# DFNN-based_predictor

In this demo, we will construct a Deep feedforward neural network(DFNN)-based predictor for predicting the acticity of CES2 inhibitor. Herein, DFNN model with were carried out by pytorch.

## Import modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
import os
import pandas as pd
import sys
import numpy as np
os.chdir('./')
#print(os.getcwd())

## Molecular characterization

In [3]:
sys.path.append('./release/') 
from Data_preprocess import load_data,calcMCFP

Load data

In [4]:
path = "pIC50数据-删除模糊数据后补充新增数据-734.csv"

dataset,canonical_smi,canonical_mols = load_data(path)

Calculate MCFP descriptors

In [5]:
pred_data = calcMCFP(mols = canonical_mols, dataset = dataset)

## Predictor construction

In [6]:
sys.path.append('./release/') 
from DFNN_predictor import RegressionDataset,data_process,Net,fit_NN_model,NN_predict

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
# 查看是否可用GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


The train_test_split from sklearn was employed for training ML model.

In [9]:
x_train,x_test,y_train,y_test,train_dataset,test_dataset = data_process(data=pred_data, 
                                                                        test_size=0.3, random_state=43)

Define hyperparameters and training NN model

In [13]:
EPOCHS = 150
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_FEATURES = x_train.shape[1] #查看行数 len(X_train) 或 X_train.shape[0]；查看列数 X_train.shape[1]

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

# training model
loss_status,model=fit_NN_model(train_loader,test_loader,EPOCHS,BATCH_SIZE,LEARNING_RATE,NUM_FEATURES)

## 十次重复实验

In [20]:
result111 = {"R2_train": [], "R2_test": [], "MSE_train": [],"MSE_test": []}

for i in range(20):
    
    
    loss_status,model=fit_NN_model(train_loader,test_loader,EPOCHS,BATCH_SIZE,LEARNING_RATE,NUM_FEATURES)
    
    y_pred_train = model(torch.from_numpy(x_train).float()).tolist()
    y_pred_test = model(torch.from_numpy(x_test).float()).tolist()
    
    result111["R2_train"].append(r2_score(y_train, y_pred_train)) #输出训练集和测试集R Square的结果
    result111["R2_test"].append(r2_score(y_test, y_pred_test))
    result111["MSE_train"].append(mean_squared_error(y_train, y_pred_train))
    result111["MSE_test"].append(mean_squared_error(y_test, y_pred_test))

Begin training.


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 38.20195 | Test Loss: 39.07507
Epoch 002: | Train Loss: 37.68136 | Test Loss: 38.12298
Epoch 003: | Train Loss: 37.21672 | Test Loss: 36.76679
Epoch 004: | Train Loss: 35.11477 | Test Loss: 33.59139
Epoch 005: | Train Loss: 30.42025 | Test Loss: 26.53878
Epoch 006: | Train Loss: 24.32776 | Test Loss: 14.38379
Epoch 007: | Train Loss: 8.92491 | Test Loss: 4.06230
Epoch 008: | Train Loss: 4.36284 | Test Loss: 6.12646
Epoch 009: | Train Loss: 4.72588 | Test Loss: 3.86982
Epoch 010: | Train Loss: 3.97870 | Test Loss: 3.41397
Epoch 011: | Train Loss: 2.93907 | Test Loss: 3.24121
Epoch 012: | Train Loss: 2.76504 | Test Loss: 3.15359
Epoch 013: | Train Loss: 2.67974 | Test Loss: 2.98160
Epoch 014: | Train Loss: 2.54245 | Test Loss: 2.83238
Epoch 015: | Train Loss: 2.51154 | Test Loss: 2.71979
Epoch 016: | Train Loss: 3.36058 | Test Loss: 2.61539
Epoch 017: | Train Loss: 2.37475 | Test Loss: 2.56897
Epoch 018: | Train Loss: 2.14767 | Test Loss: 2.35838
Epoch 019: | Tra

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 39.71754 | Test Loss: 40.03689
Epoch 002: | Train Loss: 38.35868 | Test Loss: 39.45195
Epoch 003: | Train Loss: 42.22313 | Test Loss: 38.78136
Epoch 004: | Train Loss: 42.31578 | Test Loss: 37.80727
Epoch 005: | Train Loss: 36.30861 | Test Loss: 35.92226
Epoch 006: | Train Loss: 35.07088 | Test Loss: 32.05824
Epoch 007: | Train Loss: 29.20720 | Test Loss: 24.90144
Epoch 008: | Train Loss: 22.34539 | Test Loss: 14.33786
Epoch 009: | Train Loss: 10.86532 | Test Loss: 4.43519
Epoch 010: | Train Loss: 3.59577 | Test Loss: 5.74402
Epoch 011: | Train Loss: 4.94833 | Test Loss: 3.84396
Epoch 012: | Train Loss: 4.19300 | Test Loss: 3.69101
Epoch 013: | Train Loss: 3.32780 | Test Loss: 3.29621
Epoch 014: | Train Loss: 3.08457 | Test Loss: 3.35071
Epoch 015: | Train Loss: 2.78423 | Test Loss: 3.04093
Epoch 016: | Train Loss: 2.61630 | Test Loss: 2.80810
Epoch 017: | Train Loss: 2.41835 | Test Loss: 2.65722
Epoch 018: | Train Loss: 2.32590 | Test Loss: 2.49308
Epoch 019: 

...

Epoch 001: | Train Loss: 39.20369 | Test Loss: 39.23815
Epoch 002: | Train Loss: 38.37359 | Test Loss: 38.30925
Epoch 003: | Train Loss: 40.36951 | Test Loss: 36.98740
Epoch 004: | Train Loss: 35.16481 | Test Loss: 34.69244
Epoch 005: | Train Loss: 33.04051 | Test Loss: 29.80287
Epoch 006: | Train Loss: 26.63849 | Test Loss: 20.73363
Epoch 007: | Train Loss: 16.19300 | Test Loss: 8.91144
Epoch 008: | Train Loss: 5.73908 | Test Loss: 4.35698
Epoch 009: | Train Loss: 6.07821 | Test Loss: 5.97613
Epoch 010: | Train Loss: 4.12219 | Test Loss: 3.86031
Epoch 011: | Train Loss: 3.15258 | Test Loss: 3.52292
Epoch 012: | Train Loss: 3.46817 | Test Loss: 3.38450
Epoch 013: | Train Loss: 4.60237 | Test Loss: 3.20240
Epoch 014: | Train Loss: 2.81288 | Test Loss: 3.28776
Epoch 015: | Train Loss: 2.72597 | Test Loss: 2.96372
Epoch 016: | Train Loss: 2.53012 | Test Loss: 2.78558
Epoch 017: | Train Loss: 2.50743 | Test Loss: 2.69207
Epoch 018: | Train Loss: 2.32157 | Test Loss: 2.59698
Epoch 019: | Tr

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 40.25394 | Test Loss: 38.67225
Epoch 002: | Train Loss: 36.91128 | Test Loss: 37.71722
Epoch 003: | Train Loss: 37.06013 | Test Loss: 36.14649
Epoch 004: | Train Loss: 36.00234 | Test Loss: 32.88130
Epoch 005: | Train Loss: 30.88541 | Test Loss: 26.51960
Epoch 006: | Train Loss: 22.14905 | Test Loss: 16.19399
Epoch 007: | Train Loss: 11.20919 | Test Loss: 5.62452
Epoch 008: | Train Loss: 4.20776 | Test Loss: 5.53622
Epoch 009: | Train Loss: 4.72954 | Test Loss: 4.44103
Epoch 010: | Train Loss: 3.41743 | Test Loss: 3.74424
Epoch 011: | Train Loss: 3.32202 | Test Loss: 3.54154
Epoch 012: | Train Loss: 3.03865 | Test Loss: 3.38414
Epoch 013: | Train Loss: 4.86963 | Test Loss: 3.30025
Epoch 014: | Train Loss: 3.99599 | Test Loss: 3.19315
Epoch 015: | Train Loss: 3.31689 | Test Loss: 2.88688
Epoch 016: | Train Loss: 2.53219 | Test Loss: 2.76876
Epoch 017: | Train Loss: 2.46134 | Test Loss: 2.59384
Epoch 018: | Train Loss: 2.62284 | Test Loss: 2.54520
Epoch 019: | Tr

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 40.17637 | Test Loss: 37.22912
Epoch 002: | Train Loss: 36.95506 | Test Loss: 36.23726
Epoch 003: | Train Loss: 38.67061 | Test Loss: 34.19677
Epoch 004: | Train Loss: 34.92005 | Test Loss: 29.91157
Epoch 005: | Train Loss: 26.26374 | Test Loss: 21.61025
Epoch 006: | Train Loss: 18.66861 | Test Loss: 9.71496
Epoch 007: | Train Loss: 5.90506 | Test Loss: 4.14818
Epoch 008: | Train Loss: 5.12107 | Test Loss: 5.66477
Epoch 009: | Train Loss: 3.81985 | Test Loss: 3.53258
Epoch 010: | Train Loss: 3.21642 | Test Loss: 3.58632
Epoch 011: | Train Loss: 3.20402 | Test Loss: 3.23600
Epoch 012: | Train Loss: 4.99620 | Test Loss: 3.32335
Epoch 013: | Train Loss: 2.77143 | Test Loss: 3.05965
Epoch 014: | Train Loss: 2.95113 | Test Loss: 2.78394
Epoch 015: | Train Loss: 2.38580 | Test Loss: 2.65480
Epoch 016: | Train Loss: 2.40397 | Test Loss: 2.53119
Epoch 017: | Train Loss: 3.97001 | Test Loss: 2.31813
Epoch 018: | Train Loss: 2.14311 | Test Loss: 2.43668
Epoch 019: | Trai

In [22]:
#保存结果
with open('DFNN十次重复实验.csv', 'w') as f:
    [f.write('{0},{1}\n'.format(key, value)) for key, value in result111.items()]

Visualizing NN results

In [ ]:
#""""
# 训练集/测试集预测
y_pred_train = model(torch.from_numpy(x_train).float()).tolist()
y_pred_test = model(torch.from_numpy(x_test).float()).tolist()

R_value_train = r2_score(y_train, y_pred_train) #输出训练集和测试集R Square的结果
R_value_test = r2_score(y_test, y_pred_test)
MSE_value_train = mean_squared_error(y_train, y_pred_train) 
MSE_value_test = mean_squared_error(y_test, y_pred_test)
    
print("R2：",R_value_train,R_value_test,"MSE：",MSE_value_train,MSE_value_test)
print(len(y_pred_train),len(y_pred_test))
    
fig = plt.figure(figsize=(14,10))
lw = 2
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.3)
line_x = np.linspace(0,20,50)#从(-1,1)均匀取50个点
# 预测结果散点图
ax1 = fig.add_subplot(2,2,1)
plt.scatter(y_pred_train,y_train)
ax2 = fig.add_subplot(2,2,2)
plt.scatter(y_pred_test,y_test)
# loss作图
ax3 = fig.add_subplot(2,2,3)
plt.plot(list(range(1,EPOCHS+1)),loss_status['train'])
ax4 = fig.add_subplot(2,2,4)
plt.plot(list(range(1,EPOCHS+1)),loss_status['test'])
plt.show()

Save the optimal predictor.

In [ ]:
path = "./model/DFNN.pka"

torch.save(model.state_dict(), path)

## Predicting data

In [ ]:
sys.path.append('./release/') 
from DFNN_predictor import NN_predict

Input data

In [ ]:
path = "预测活性.csv"

dataset,canonical_smi,canonical_mols = load_data(path)
vali_data = calcMCFP(mols = canonical_mols, dataset = dataset)

Loading predictor

In [ ]:
data = vali_data  # validation set
path = "./model/DFNN.pka"

pre_act = NN_predict(data = vali_data, path =path, num_features = 128)